In [2]:
import numpy as np
import pandas as pd
import random
import scipy.optimize
import itertools

In [3]:
def m1(M,q):
    comp_m1 = M/(1+q)
    return comp_m1

def m2(M,q):
    comp_m2 = q*M/(1+q)
    return comp_m2

def M(m1,m2):
    return m1+m2

def v_crit(m11,m12,m21,m22,a1,a2):
    # G=1
    return np.sqrt(((M(m11,m12)+M(m21,m22)) / (M(m11,m12)*M(m21,m22))) * ((m11*m12/a1)+(m21*m22/a2)))

def b_max(vinf_vcrit,a_max):
    return (4./vinf_vcrit + 3)*a_max   # note: v is v_inf/v_crit

In [74]:
a

100.0

In [101]:
# model system, and grid we will use for varying parameters
mdl_sys={}
mdl_sys['v'] = 0.01    # v_inf/v_crit
mdl_sys['m'] = 20.0   # Msun
mdl_sys['a'] = 1.0   # AU
mdl_sys['e'] = 0.0
mdl_sys['alpha'] = 1.0  # a2/a1
mdl_sys['q'] = 1.0      # m2/m1
for key, val in zip(mdl_sys.keys(), mdl_sys.values()):
    mdl_sys[key] = np.asarray([val])

# grids
gs = 50   # grid size
a_grid = np.logspace(-2,2,gs)
alpha_grid = np.logspace(-2,2,gs)
e1_grid = np.linspace(0.0,0.98,gs)
e2_grid = np.linspace(0.0,0.98,gs)
q_grid = np.logspace(np.log10(0.03),np.log10(40),gs)
v_grid = np.logspace(-2,2,gs)

# specify which parameter we want to vary
q1 = mdl_sys['q']
q2 = mdl_sys['q']
alpha = mdl_sys['alpha']

v = mdl_sys['v']
m11 = mdl_sys['m']
m12 = mdl_sys['m'] * q1
a1 = a_grid
e1 = mdl_sys['e']
m21 = mdl_sys['m']
m22 = mdl_sys['m'] * q2
a2 = a_grid * mdl_sys['alpha']
e2 = mdl_sys['e']


dat = list(itertools.product(v, m11, m12, a1, e1, m21, m22, e2))
dat = np.asarray(dat)
df = pd.DataFrame(dat,columns=['v_inf/v_crit','m11','m12','a1','e1','m21','m22','e2'])

# for the grid over a...
df['a2'] = df['a1']

In [92]:
# 4-body
grid_name = 'SMA_20M_0.01V_binbin'
grid_path = 'grid_files/'+grid_name+'.dat'
gridinfo_path = 'grid_files/'+grid_name+'_info.txt'

f = open(gridinfo_path, 'w')
f.write('GRID: N_points=%i \n q1: %f-%f \n q2: %f-%f \n alpha: %f-%f \n vinf_crit: %f-%f \n \
m11: %f-%f \n m12: %f-%f \n a1: %f-%f \n e1: %f-%f \n \
m21: %f-%f \n m22: %f-%f \n a2: %f-%f \n e2: %f-%f' % \
       (gs,min(q1),max(q1),min(q2),max(q2),min(alpha),max(alpha),min(v),max(v),\
        min(m11),max(m11),min(m12),max(m12),min(a1),max(a1),min(e1),max(e1), \
        min(m21),max(m21),min(m22),max(m22),min(a2),max(a2),min(e2),max(e2)))
f.close()

f = open(grid_path, 'w')
f.write('1:b/(a1+a2) 2:v/v_crit 3:m11(msun) 4:m12 5:a1(AU) 6:e1 7:m21 8:m22 9:a2 10:e2 11:n_exchange 12:n_s 13:n_b 14:n_b 15:t 16:r')
f.close()

for i in xrange(len(dat)):
    a1 = df['a1'].iloc[i]
    a2 = df['a2'].iloc[i]
    e1 = df['e1'].iloc[i]
    e2 = df['e2'].iloc[i]
    m11 = df['m11'].iloc[i]
    m12 = df['m12'].iloc[i]
    m21 = df['m21'].iloc[i]
    m22 = df['m22'].iloc[i]
    v_vcrit = df['v_inf/v_crit'].iloc[i]
    # NOTE: gotta check this...also can use (a1+a2)
    if a1>=a2:
        b = b_max(v_vcrit,a1)
    else:
        b = b_max(v_vcrit,a2)
    f = open(grid_path, 'a')
    f.write('\n%f %f %f %f %f %f %f %f %f %f 0 0 0 0 0 0' % \
            (b/(a1+a2),v_vcrit,m11,m12,a1,e1,m21,m22,a2,e2))
    f.close()
    

In [102]:
# 3-body   
grid_name = 'SMA_20M_0.01V_binsin'
grid_path = 'grid_files/'+grid_name+'.dat'
gridinfo_path = 'grid_files/'+grid_name+'_info.txt'

f = open(gridinfo_path, 'w')
f.write('GRID: N_points=%i \n q: %f-%f \n vinf_crit: %f-%f \n \
m11: %f-%f \n m12: %f-%f \n a1: %f-%f \n e1: %f-%f \n m21: %f-%f' % \
       (gs,min(q),max(q),min(v),max(v),\
        min(m11),max(m11),min(m12),max(m12),min(a1),max(a1),min(e1),max(e1), \
        min(m21),max(m21)))
f.close()

f = open(grid_path, 'w')
f.write('1:b/a 2:v/v_crit 3:m1(msun) 4:m2 5:a(AU) 6:e 7:m_s 8:n_exchange 9:n_s 10:n_b 11:n_b 12:t 13:r')
f.close()

for i in xrange(len(dat)):
    a = df['a1'].iloc[i]
    e = df['e1'].iloc[i]
    m11 = df['m11'].iloc[i]
    m12 = df['m12'].iloc[i]
    m21 = df['m21'].iloc[i]
    v_vcrit = df['v_inf/v_crit'].iloc[i]
    b = b_max(v_vcrit,a)
    f = open(grid_path, 'a')
    f.write('\n%f %f %f %f %f %f %f 0 0 0 0 0 0' % \
            (b/a,v_vcrit,m11,m12,a,e,m21))
    f.close()